In [38]:
import numpy as np 
import scipy
from scipy.io import loadmat
import pandas as pd
import tensorflow as tf 
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, cohen_kappa_score, hamming_loss
from sklearn.model_selection import train_test_split
import random 
import matplotlib.pyplot as plt
random.seed = 0

import warnings
warnings.filterwarnings('ignore')


In [2]:
stack_chemistry = loadmat("/home/karim/Documents/research/sourceCode/MLML/mlml_weightedLoss/Datasets/junHuang/stack/Stackex_chemistry.mat")

features = stack_chemistry['data']
features_df = pd.DataFrame(features)

labels = stack_chemistry['target'].T
labels_df = pd.DataFrame(labels)
print("Number of training samples is: {}".format(len(features)))
print("Ratio of positive labels is: {:.2f}%".format(100 * labels.sum()/(labels.shape[0]*labels.shape[1])))

#LABEL_LIST = list(features_labels_df.columns[-174:])

features, test_features, labels, test_labels = train_test_split(features, labels, test_size=0.33, random_state=0)

Number of training samples is: 6961
Ratio of positive labels is: 1.21%


## NN approach

In [3]:
input_shape = features.shape[1]
output_shape = labels.shape[1]
hidden_layer_1_shape = 240
hidden_layer_2_shape = 120

def get_weights(shape):
    w = tf.Variable(tf.truncated_normal(shape, stddev=0.1))
    #variable_summaries(w)
    return w

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    b = tf.Variable(initial)
    #variable_summaries(b)
    return b

def full_layer(input, size):
    in_size = int(input.get_shape()[1])
    W = get_weights([in_size, size])
    b = bias_variable([size])
    return tf.matmul(input, W) + b

def weighted_loss(y_true, y_pred, positive_weights, negative_weights):
    # clip to prevent NaN's and Inf's
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1-1e-7, name=None)
    #y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # calc
    loss = (-y_true * tf.log(y_pred) * positive_weights) - ((1.0 - y_true) * tf.log(1.0 - y_pred) * negative_weights)
    loss = tf.reduce_mean(loss)
    return loss

In [4]:
# Define a 2 layers network to train 
y = tf.placeholder(tf.float32, [None, output_shape], name="true_labels")
x_input = tf.placeholder(tf.float32, [None,input_shape],name="input_layer")
h1 = tf.nn.tanh(full_layer(x_input, hidden_layer_1_shape))
h2 = tf.nn.tanh(full_layer(h1, hidden_layer_2_shape))
#h3 = tf.nn.relu(full_layer(h2, hidden_layer_2_shape))
#h4 = tf.nn.relu(full_layer(h3, hidden_layer_2_shape))
logits = full_layer(h2,output_shape)
output = tf.nn.sigmoid(logits)

## Train on complete dataset

In [25]:
# using weighted cross entropy because dataset is sparse and we need to weight positives more
#loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y))
loss = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(logits=logits, labels=y,pos_weight = 10))


# Learning rate decay
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(learning_rate=0.1, global_step=global_step, decay_steps=1000,
                                          decay_rate=0.95,staircase=True)
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
correct_prediction = tf.equal(tf.round(output), y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [26]:
NUM_EPOCHS = 10000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(NUM_EPOCHS):
        epoch_loss, epoch_accuracy,epoch_output, _ = sess.run([loss, accuracy,output, train_step],feed_dict={x_input: 
                                                                                         features,y: labels,})
        if (epoch+1)% 500 == 0:
            val_losses, val_accuracies, val_output,current_learning_rate = sess.run([loss, accuracy,output,learning_rate],feed_dict={
                                                                                          x_input: test_features,
                                                                                          y:test_labels})
            print("Epoch #{}".format(epoch+1), "Loss: {:.4f}".format(epoch_loss), 
                  "accuracy: {:.4f}".format(epoch_accuracy),
                  "Test loss: {:.4f}".format(val_losses), 
                  "Test accuracy: {:.4f}".format(val_accuracies))
            if (epoch+1) == NUM_EPOCHS:
                print("====================================== \
                      \n\nFinal test accuracy: {:.4f}".format(val_accuracies))

Epoch #500 Loss: 0.3210 accuracy: 0.9734 Test loss: 0.3198 Test accuracy: 0.9732
Epoch #1000 Loss: 0.2951 accuracy: 0.9746 Test loss: 0.2948 Test accuracy: 0.9744
Epoch #1500 Loss: 0.2881 accuracy: 0.9751 Test loss: 0.2889 Test accuracy: 0.9748
Epoch #2000 Loss: 0.2821 accuracy: 0.9756 Test loss: 0.2839 Test accuracy: 0.9752
Epoch #2500 Loss: 0.2762 accuracy: 0.9760 Test loss: 0.2790 Test accuracy: 0.9758
Epoch #3000 Loss: 0.2703 accuracy: 0.9765 Test loss: 0.2742 Test accuracy: 0.9762
Epoch #3500 Loss: 0.2649 accuracy: 0.9769 Test loss: 0.2697 Test accuracy: 0.9767
Epoch #4000 Loss: 0.2597 accuracy: 0.9773 Test loss: 0.2655 Test accuracy: 0.9770
Epoch #4500 Loss: 0.2551 accuracy: 0.9775 Test loss: 0.2619 Test accuracy: 0.9772
Epoch #5000 Loss: 0.2507 accuracy: 0.9777 Test loss: 0.2585 Test accuracy: 0.9773
Epoch #5500 Loss: 0.2468 accuracy: 0.9778 Test loss: 0.2555 Test accuracy: 0.9773
Epoch #6000 Loss: 0.2431 accuracy: 0.9779 Test loss: 0.2528 Test accuracy: 0.9773
Epoch #6500 Loss:

In [27]:
# On training 
print("AUC is:{:.3f}".format(roc_auc_score(labels, epoch_output)))
print("F1 is:{:.3f}".format(f1_score(labels, np.round(epoch_output),average="macro")))
print("Recall is:{:.3f}".format(recall_score(labels, np.round(epoch_output),average="macro")))
print("Precision is:{:.3f}".format(precision_score(labels, np.round(epoch_output),average="macro")))
print("Hamming loss is:{:.3f}".format(hamming_loss(labels, np.round(epoch_output))))

AUC is:0.840
F1 is:0.099
Recall is:0.119
Precision is:0.159
Hamming loss is:0.022


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:
# Adjusting threshold 
thresholds = np.arange(0, 1, 0.01)
f1_array = np.zeros((output_shape, len(thresholds)))
for idx in range(output_shape):
    f1_array[idx, :] = [
        f1_score(labels[:, idx], np.clip(np.round(epoch_output[:, idx] - threshold + 0.5), 0, 1))
        for threshold in thresholds]
threshold_arg = np.argmax(f1_array, axis=1)
threshold_per_class = thresholds[threshold_arg]

# Applying thresholds optimized per class
model_output_rounded = np.zeros_like(epoch_output)
for idx in range(output_shape):
    model_output_rounded[:, idx] = np.clip(np.round(epoch_output[:, idx] - threshold_per_class[idx] + 0.5), 0, 1)

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
print("F1 is:{:.3f}".format(f1_score(labels, model_output_rounded,average="macro")))
print("Recall is:{:.3f}".format(recall_score(labels, model_output_rounded,average="macro")))
print("Precision is:{:.3f}".format(precision_score(labels, model_output_rounded,average="macro")))
print("Hamming loss is:{:.3f}".format(hamming_loss(labels, model_output_rounded)))

F1 is:0.207
Recall is:0.312
Precision is:0.245
Hamming loss is:0.052


In [30]:
# Applying thresholds optimized per class for testset
test_output_rounded = np.zeros_like(val_output)
for idx in range(output_shape):
    test_output_rounded[:, idx] = np.clip(np.round(val_output[:, idx] - threshold_per_class[idx] + 0.5), 0, 1)

In [33]:
# On test
#print("AUC is:{:.3f}".format(roc_auc_score(test_labels, val_output)))
print("F1 is:{:.3f}".format(f1_score(test_labels, np.round(val_output),average="macro")))
print("Recall is:{:.3f}".format(recall_score(test_labels, np.round(val_output),average="macro")))
print("Precision is:{:.3f}".format(precision_score(test_labels, np.round(val_output),average="macro")))
print("Hamming loss is:{:.3f}".format(hamming_loss(test_labels, np.round(val_output))))
print("After treshold optimization")
print("F1 is:{:.3f}".format(f1_score(test_labels, test_output_rounded,average="macro")))
print("Recall is:{:.3f}".format(recall_score(test_labels,test_output_rounded,average="macro")))
print("Precision is:{:.3f}".format(precision_score(test_labels, test_output_rounded,average="macro")))
print("Hamming loss is:{:.3f}".format(hamming_loss(test_labels, test_output_rounded)))

F1 is:0.070
Recall is:0.091
Precision is:0.093
Hamming loss is:0.023
After treshold optimization
F1 is:0.103
Recall is:0.162
Precision is:0.104
Hamming loss is:0.055


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## With missing labels

In [34]:
ones_indices = np.nonzero(labels)
ratio_of_hidden_samples = 0.4
number_of_hidden_samples = int(len(ones_indices[0]) * ratio_of_hidden_samples)
random_indices = random.sample(list(np.arange(len(ones_indices[0]))),number_of_hidden_samples)
indices_to_hide = (ones_indices[0][random_indices] , ones_indices[1][random_indices])
labels_with_missing_positives = np.copy(labels)
for counter in range (number_of_hidden_samples):
    labels_with_missing_positives[indices_to_hide[0][counter]][indices_to_hide[1][counter]] = 0
    
    
# Training with missing labels with 40%
NUM_EPOCHS = 5000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(NUM_EPOCHS):
        epoch_loss, epoch_accuracy,epoch_output, _ = sess.run([loss, accuracy,output, train_step],feed_dict={x_input: 
                                                                                         features,y: labels_with_missing_positives})
        if (epoch+1)% 500 == 0:
            val_losses, val_accuracies, val_output,current_learning_rate = sess.run([loss, accuracy,output,learning_rate],feed_dict={
                                                                                          x_input: test_features,
                                                                                          y:test_labels})
            print("Epoch #{}".format(epoch+1), "Loss: {:.4f}".format(epoch_loss), 
                  "accuracy: {:.4f}".format(epoch_accuracy), 
                  "Test loss: {:.4f}".format(val_losses), 
                  "Test accuracy: {:.4f}".format(val_accuracies))

Epoch #500 Loss: 0.2375 accuracy: 0.9880 Test loss: 0.3182 Test accuracy: 0.9842
Epoch #1000 Loss: 0.2139 accuracy: 0.9879 Test loss: 0.3042 Test accuracy: 0.9841
Epoch #1500 Loss: 0.2093 accuracy: 0.9880 Test loss: 0.3015 Test accuracy: 0.9842
Epoch #2000 Loss: 0.2064 accuracy: 0.9880 Test loss: 0.2990 Test accuracy: 0.9844
Epoch #2500 Loss: 0.2038 accuracy: 0.9881 Test loss: 0.2964 Test accuracy: 0.9846
Epoch #3000 Loss: 0.2012 accuracy: 0.9882 Test loss: 0.2934 Test accuracy: 0.9847
Epoch #3500 Loss: 0.1986 accuracy: 0.9883 Test loss: 0.2904 Test accuracy: 0.9849
Epoch #4000 Loss: 0.1961 accuracy: 0.9883 Test loss: 0.2874 Test accuracy: 0.9851
Epoch #4500 Loss: 0.1937 accuracy: 0.9883 Test loss: 0.2846 Test accuracy: 0.9851
Epoch #5000 Loss: 0.1913 accuracy: 0.9883 Test loss: 0.2819 Test accuracy: 0.9851


In [ ]:
# Adjusting threshold 
thresholds = np.arange(0, 1, 0.01)
f1_array = np.zeros((output_shape, len(thresholds)))
for idx in range(output_shape):
    f1_array[idx, :] = [
        f1_score(labels[:, idx], np.clip(np.round(epoch_output[:, idx] - threshold + 0.5), 0, 1))
        for threshold in thresholds]
threshold_arg = np.argmax(f1_array, axis=1)
threshold_per_class = thresholds[threshold_arg]

# Applying thresholds optimized per class
model_output_rounded = np.zeros_like(epoch_output)
for idx in range(output_shape):
    model_output_rounded[:, idx] = np.clip(np.round(epoch_output[:, idx] - threshold_per_class[idx] + 0.5), 0, 1)

In [39]:
# On training 
print("Training set evaluation")
print("AUC is:{:.3f}".format(roc_auc_score(labels, epoch_output)))
print("F1 is:{:.3f}".format(f1_score(labels, np.round(epoch_output),average="macro")))
print("Recall is:{:.3f}".format(recall_score(labels, np.round(epoch_output),average="macro")))
print("Precision is:{:.3f}".format(precision_score(labels, np.round(epoch_output),average="macro")))
print("Hamming loss is:{:.3f}".format(hamming_loss(labels, np.round(epoch_output))))
print("F1 is:{:.3f}".format(f1_score(labels, model_output_rounded,average="macro")))
print("Recall is:{:.3f}".format(recall_score(labels, model_output_rounded,average="macro")))
print("Precision is:{:.3f}".format(precision_score(labels, model_output_rounded,average="macro")))
print("Hamming loss is:{:.3f}".format(hamming_loss(labels, model_output_rounded)))
 
    
# Applying thresholds optimized per class for testset
test_output_rounded = np.zeros_like(val_output)
for idx in range(output_shape):
    test_output_rounded[:, idx] = np.clip(np.round(val_output[:, idx] - threshold_per_class[idx] + 0.5), 0, 1)

# On test
print("===================")
print("Test set evaluation")
#print("AUC is:{:.3f}".format(roc_auc_score(test_labels, val_output)))
print("F1 is:{:.3f}".format(f1_score(test_labels, np.round(val_output),average="macro")))
print("Recall is:{:.3f}".format(recall_score(test_labels, np.round(val_output),average="macro")))
print("Precision is:{:.3f}".format(precision_score(test_labels, np.round(val_output),average="macro")))
print("Hamming loss is:{:.3f}".format(hamming_loss(test_labels, np.round(val_output))))
print("After treshold optimization")
print("F1 is:{:.3f}".format(f1_score(test_labels, test_output_rounded,average="macro")))
print("Recall is:{:.3f}".format(recall_score(test_labels,test_output_rounded,average="macro")))
print("Precision is:{:.3f}".format(precision_score(test_labels, test_output_rounded,average="macro")))
print("Hamming loss is:{:.3f}".format(hamming_loss(test_labels, test_output_rounded)))

Training set evaluation
AUC is:0.688
F1 is:0.021
Recall is:0.020
Precision is:0.056
Hamming loss is:0.015
F1 is:0.096
Recall is:0.343
Precision is:0.104
Hamming loss is:0.187
Test set evaluation
F1 is:0.017
Recall is:0.017
Precision is:0.045
Hamming loss is:0.015
After treshold optimization
F1 is:0.061
Recall is:0.274
Precision is:0.054
Hamming loss is:0.190


## With fixed negative weights

In [43]:
train_negative_weights = np.zeros_like(labels) + 1 
train_positive_weights = np.zeros_like(labels) + 10 # Due to imbalance increase wight of positive labels
for counter in range (number_of_hidden_samples):
    train_negative_weights[indices_to_hide[0][counter]][indices_to_hide[1][counter]] = 0

In [44]:
positive_weights = tf.placeholder(tf.float32, [None,output_shape], name = "Positive_weights")
negative_weights = tf.placeholder(tf.float32, [None, output_shape], name="negative_weights")
my_weights_loss = weighted_loss(y_true= y, y_pred= output,
                              positive_weights= positive_weights, negative_weights= negative_weights)

train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(my_weights_loss,global_step=global_step)

In [45]:
# Training with negative weights!
NUM_EPOCHS = 10000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(NUM_EPOCHS):
        epoch_my_weights_loss, epoch_loss, epoch_accuracy,epoch_output, _ = sess.run([my_weights_loss, loss, accuracy,output, train_step],feed_dict={x_input: 
                                                                                         features,y: labels_with_missing_positives,positive_weights: train_positive_weights,
                                                                                                  negative_weights: train_negative_weights})
        if (epoch+1)% 500 == 0:
            val_losses, val_accuracies, val_output,current_learning_rate = sess.run([loss, accuracy,output,learning_rate],feed_dict={
                                                                                          x_input: test_features,
                                                                                          y:test_labels})
            print("Epoch #{}".format(epoch+1), "Loss: {:.4f}".format(epoch_loss), 
                  "Weighted Loss: {:.4f}".format(epoch_my_weights_loss),"accuracy: {:.4f}".format(epoch_accuracy), 
                  "Test loss: {:.4f}".format(val_losses), "Test accuracy: {:.4f}".format(val_accuracies))

Epoch #500 Loss: 0.2394 Weighted Loss: 0.2379 accuracy: 0.9866 Test loss: 0.3189 Test accuracy: 0.9829
Epoch #1000 Loss: 0.2141 Weighted Loss: 0.2126 accuracy: 0.9869 Test loss: 0.3037 Test accuracy: 0.9833
Epoch #1500 Loss: 0.2094 Weighted Loss: 0.2080 accuracy: 0.9871 Test loss: 0.3010 Test accuracy: 0.9835
Epoch #2000 Loss: 0.2065 Weighted Loss: 0.2050 accuracy: 0.9873 Test loss: 0.2987 Test accuracy: 0.9837
Epoch #2500 Loss: 0.2040 Weighted Loss: 0.2025 accuracy: 0.9875 Test loss: 0.2963 Test accuracy: 0.9839
Epoch #3000 Loss: 0.2014 Weighted Loss: 0.1999 accuracy: 0.9877 Test loss: 0.2936 Test accuracy: 0.9842
Epoch #3500 Loss: 0.1989 Weighted Loss: 0.1973 accuracy: 0.9879 Test loss: 0.2908 Test accuracy: 0.9844
Epoch #4000 Loss: 0.1964 Weighted Loss: 0.1947 accuracy: 0.9880 Test loss: 0.2880 Test accuracy: 0.9845
Epoch #4500 Loss: 0.1940 Weighted Loss: 0.1923 accuracy: 0.9881 Test loss: 0.2853 Test accuracy: 0.9847
Epoch #5000 Loss: 0.1917 Weighted Loss: 0.1899 accuracy: 0.9881 T

# XGBoost

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

175